In [1]:
#region Selenium Controllers

#!conda install -y selenium
#!pip install webdriver_manager

import argparse
from enum import Enum
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains
import time
import random
from webdriver_manager.firefox import GeckoDriverManager

driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())




def waitFinishLoad(driver,idLoader,needsDisapeared=False,by=By.ID,needsWait=True):
    """Espera a la carga de un elemento antes de continuar

    Args:
        driver (WebDriver): WebDriver
        idLoader (str): Id a buscar, puede ser XPath...
        needsDisapeared (bool, optional): Si el elemento a esperar tiene que aparecer o desaparecer como un CircularProgress. Defaults to False.
        by (str, optional): Tipo de By a buscar el Wait. Defaults to By.ID.
        needsWait (bool, optional): Si necesita esperar un extra para que no pueda saltar un Captcha. Defaults to False.
    """
    if idLoader is None:
        return
    WebDriverWait(driver, timeout=60).until(EC.presence_of_element_located((by, idLoader)))
    if needsDisapeared:
        WebDriverWait(driver, timeout=60).until(EC.invisibility_of_element((by, idLoader)))
    if needsWait:
        time.sleep(random.choice([0.05,0.1,0.2]))

def elementBy(query,driver,**kargs):
    """Devuelve un element|None dependiendo de la consulta

    Args:
        query (str): Busqueda a realizar del elemento
        driver (WebDriver): WebDriver

    Returns:
        Element|None: Elemento de la búsqueda
    """
    if not kargs.get("needsWait",False):
        waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
        element=driver.find_element(kargs.get("by",By.XPATH),query)
        try:
            driver.execute_script("arguments[0].scrollIntoView();", element)
        except:
            None
    waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    return element

def clickEvent(driver,element,**kargs):
    """Click en un elemento que funciona con evento

    Args:
        driver (WebDriver): WebDriver
        element (Element): Elemento que tiene el evento
    """
    if not kargs.get("needsWait",False):
        waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
    if kargs.get("normal",False):
        element.click
    else:
        try:
            driver.execute_script(element.get_dom_attribute("onClick"))
        except:
            element.click
    waitFinishLoad(driver,kargs.get("idLoader",None),by=kargs.get("by",By.ID),needsWait=kargs.get("needsWait",False))
# endregion

TypeError: __init__() got an unexpected keyword argument 'service'

In [ ]:
URL="https://supermarioplay.com/"

In [ ]:
import threading
from time import sleep
from selenium.webdriver.common.keys import Keys

def control_player(points,threadList):
    actions = webdriver.ActionChains(driver)
    distancia = devolver_distancia(points[3],points[4])
    if distancia[0] > 0 and not threadList[0]:
        threadList[0]=threading.Thread(target=movimiento,args=(actions,Keys.ARROW_RIGHT,0.01,threadList,0))
        threadList[0].start()
    if distancia[0] < 0 and not threadList[1]:
        threadList[1]=threading.Thread(target=movimiento,args=(actions,Keys.ARROW_LEFT,0.01,threadList,1))
        threadList[1].start()
    if distancia[1] < 0 and not threadList[2]:
        threadList[2]=threading.Thread(target=movimiento,args=(actions,Keys.ARROW_UP,1,threadList,2))
        threadList[2].start()
    #if distancia[1] > 0 and not threadList[3]:
    #    threadList[3]=threading.Thread(target=movimiento,args=(actions,Keys.ARROW_DOWN,1,threadList,3))
    #    threadList[3].start()
    return threadList

def devolver_distancia(p1,p2):
    return p2.x-p1.x,p2.y-p1.y

def movimiento(actions,key,time,lista,pos):
    actions.key_down(key).perform()
    sleep(time)
    actions.key_up(key).perform()
    lista[pos]=None

In [ ]:
#@markdown We implemented some functions to visualize the hand landmark detection results. <br/> Run the following cell to activate the functions.

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

MARGIN = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
HANDEDNESS_TEXT_COLOR = (88, 205, 54) # vibrant green

def draw_landmarks_on_image(rgb_image, detection_result):
  hand_landmarks_list = detection_result.hand_landmarks
  handedness_list = detection_result.handedness
  annotated_image = np.copy(rgb_image)

  # Loop through the detected hands to visualize.
  for idx in range(len(hand_landmarks_list)):
    hand_landmarks = hand_landmarks_list[idx]
    handedness = handedness_list[idx]

    # Draw the hand landmarks.
    hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    hand_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks
    ])
    solutions.drawing_utils.draw_landmarks(
      annotated_image,
      hand_landmarks_proto,
      solutions.hands.HAND_CONNECTIONS,
      solutions.drawing_styles.get_default_hand_landmarks_style(),
      solutions.drawing_styles.get_default_hand_connections_style())

    # Get the top left corner of the detected hand's bounding box.

    # Draw handedness (left or right hand) on the image.

  return hand_landmarks_list,annotated_image

In [ ]:
import pathlib
import time
import cv2 as cv
from matplotlib import pyplot as plt
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

path=str(pathlib.Path().resolve())+"/"


options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=path+"hand_landmarker.task"),
    running_mode=VisionRunningMode.VIDEO,
    num_hands=1,)
t = time.time()
video = cv.VideoCapture(0)
threadList=[None,None,None,None]
with HandLandmarker.create_from_options(options) as landmarker:
    while(True):
        ret, frame = video.read()
        if ret == True:
            frame = cv.flip(frame, 1)
            h, w, c = frame.shape
            mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        
            hand_landmarker_result = landmarker.detect_for_video(mp_image,int((time.time()-t)*1000))

            if driver.current_url!=URL:
                driver.get(URL)
                waitFinishLoad(driver,"//iframe",by=By.XPATH)
                elementBy("//*[text()='Consentir']",driver,by=By.XPATH,needsDisapeared=True).click()
                juego=elementBy("//iframe",driver,by=By.XPATH).click()
            # Si tiene gestos los detecta si no lo transforma en numpy image
            if len(hand_landmarker_result.hand_landmarks)>0:
                points,mp_image=draw_landmarks_on_image(mp_image.numpy_view(), hand_landmarker_result)
                control_player(points[0],threadList)
            else:
                mp_image = mp_image.numpy_view()

            cv.imshow("Result", cv.cvtColor(mp_image,cv.COLOR_BGR2RGB))
        if ret == False:
            video.set(cv.CAP_PROP_POS_FRAMES, 0)
        if cv.waitKey(10) & 0xFF == 27: break

video.release()
cv.destroyAllWindows()